GRUPO 1 : Alejandro Ramírez, Mario Jimenez, David Seijas

## Tercera parte.  Recomendacion basada en filtrado colaborativo.

En esta tercera parte utilizaremos la librería SURPRISE Se puede consultar la documentacion en http://surpriselib.com/

Para instalarla: conda install -c conda-forge scikit-surprise o pip install numpy pip install scikit-surprise

La librería SurPRISE (Simple Python RecommendatIon System Engine) tiene algoritmos de predición de ratings incluidos: baseline algorithms, neighborhood methods, matrix factorization-based ( SVD, PMF, SVD++, NMF) y otros. También tiene predefinidas las medidas de similitud mas comunes sobre vectores (cosine, MSD, pearson…) Una de las cosas más utiles es que proporciona herramientas para evaluar, analizar y comparar el rendimiento de distitnos algoritmos. Lo que vamos a hacer en esta parte de la práctica es probar varios procedimientos de evaluación cruzada midiendo datos sobre errores entre el valor real (conocido) y la predicción del recomendador. Las siglas corresponden a las siguientes medidas:

MAE: Mean Absolute Error
RMSE: Root mean square error (RMSE)
MSE: mean square error is defined as the expected value of the square of the difference between the estimator and the parameter. -square root of the mean square error.

Vamos a ejecutar algunos recomendadores y evaluarlos para poder comentar los resultados.


In [1]:
 conda install -c conda-forge scikit-surprise 

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [19]:
from collections import defaultdict
import numpy as np

from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import SVDpp
from surprise import NMF

from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [3]:
## Ejemplo getting started de la documentación de SURPRISE
##http://surpriselib.com/

from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)



Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9413  0.9339  0.9325  0.9393  0.9409  0.9376  0.0036  
MAE (testset)     0.7427  0.7371  0.7348  0.7377  0.7411  0.7387  0.0029  
Fit time          4.97    5.01    5.03    5.00    5.02    5.01    0.02    
Test time         0.18    0.14    0.18    0.14    0.18    0.16    0.02    


{'test_rmse': array([0.94127105, 0.93393388, 0.93251735, 0.93932072, 0.94086956]),
 'test_mae': array([0.7427485 , 0.73712058, 0.73482633, 0.73770209, 0.74111342]),
 'fit_time': (4.969994783401489,
  5.006847858428955,
  5.028186082839966,
  5.003286123275757,
  5.02266788482666),
 'test_time': (0.1753091812133789,
  0.14464950561523438,
  0.1809232234954834,
  0.13762760162353516,
  0.17706298828125)}

In [4]:
# Evaluacion extracted from surprise: 
# https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-compute-precision-k-and-recall-k
def measures_at_k(predictions, k, th_recom, th_relev):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    onehits = dict()
    mrr = dict()
    
    for uid, user_ratings in user_est_true.items():
        
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= th_relev) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= th_recom) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= th_relev) and (est >= th_recom))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
       
        
    return precisions, recalls


In [5]:
def f1(precision, recall):
    """
        Funcion que calcula el f1 (media armónica) en funcion de precision y recall
    """
    denominador = precision + recall
    
    if denominador == 0:
        return 0
    else:
        return (2 * precision * recall) / denominador

In [6]:
def get_results(recommendations, k, knn):
    """
        Function to get the measures results 
    """
    # threshold = 4 --> solo se tienen en cuenta peliculas que hayan 
    # sido puntuadas con 4 o 5 estrellas
    precisions, recalls  = measures_at_k(recommendations, k, th_recom=4, th_relev=1)
    
    # Measures can then be averaged over all users
    precision_result = sum(prec for prec in precisions.values()) / len(precisions)
    recall_result = sum(rec for rec in recalls.values()) / len(recalls)
    # Media armónica  
    f1_result = f1(precision_result, recall_result)
    # En este archivo se volcarán los resultados de las métricas. Tiene que existir. 
    f = open("C:/Users/aleja/Google Drive/4o/INFORMATICA/INTELIGENCIA ARTIFICIAL/PRACTICAS/PR2/results_user_cf.csv", 'a')
    #f = open("C:/hlocal/results_user_cf.csv", 'a')
    f.write(str(k) + ',' + knn + "," + str(precision_result) + ',' + str(recall_result) + ',' +  str(f1_result) +  '\n') 
    f.close()
    

In [7]:
# Hemos cargado antes los datos de movieLens para 100K
# data = Dataset.load_builtin('ml-100k')

In [8]:
# creo dos conjuntos de datos, el training set y el evaluation set
# cada uno contendra la mitad de los datos
training_set, evaluation_set = train_test_split(data, test_size=.5)

In [9]:
# Ahora determino cual es el algoritmo que voy a usar de recomendacion
# en este caso voy a usar el algoritmo KNN para encontrar las similitudes entre items
recommendation_algorithm = KNNBasic(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})
#print(recommendation_algorithm)

In [10]:
# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)
#print(recommendation_algorithm)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [11]:
# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)
#print(recommendations)

In [13]:
K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_basic")

In [14]:
##########################################################
# Hacer distintas pruebas con el resto de tipos KNN
recommendation_algorithm = KNNWithMeans(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_withmeans")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [15]:
##########################################################
# Hago lo mismo con el resto de tipos KNN
recommendation_algorithm = KNNWithZScore(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_withzscore")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [16]:
##########################################################
# Hago lo mismo con el resto de tipos KNN
recommendation_algorithm = KNNBaseline(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_baseline")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [18]:
# Hago lo mismo con el resto de tipos SVD
recommendation_algorithm = SVD()

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "SVD")

In [ ]:
# Hago lo mismo con el resto de tipos SVDpp
recommendation_algorithm = SVDpp()

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "SVDpp")

In [21]:
# Hago lo mismo con el resto de tipos NMF
recommendation_algorithm = NMF()

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "NMF")

In [23]:
# Hago lo mismo con el resto de tipos SVD
recommendation_algorithm = SVD(n_factors=150,n_epochs=25)

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "SVD")

In [24]:
# Hago lo mismo con el resto de tipos SVDpp
recommendation_algorithm = SVDpp(n_factors=70,n_epochs=25)

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "SVDpp")

In [25]:
# Hago lo mismo con el resto de tipos NMF
recommendation_algorithm = NMF(n_factors=30,n_epochs=80)

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "NMF")

## Ejercicio:  se pide ejecutar, comprender y escribir comentarios razonados sobre la evaluación de distintos recomendadores.
    
Prueba otros algoritmos de predicción de ratings basados en la estimación de los vecinos más próximos y realiza evaluaciones de su comportamiento. Comenta los resultados.¶
Puedes consultar la documentación en https://surprise.readthedocs.io/en/stable/knn_inspired.html#

In [ ]:
1	knn_basic	0.938494168	0.035488692	0.068391205
2	knn_basic	0.938494168	0.065003351	0.121585285
3	knn_basic	0.938494168	0.088511622	0.161766645
4	knn_basic	0.938494168	0.107057289	0.192190715
5	knn_basic	0.938494168	0.122343821	0.216468422
6	knn_basic	0.938494168	0.135060745	0.23613831
7	knn_basic	0.938494168	0.146200698	0.252990047
8	knn_basic	0.938494168	0.156314649	0.267992701
9	knn_basic	0.938494168	0.16478449	0.280344919
10	knn_basic	0.938494168	0.172270338	0.291105283

1	knn_withmeans	0.841993637	0.03131557	0.060385281
2	knn_withmeans	0.841993637	0.058728259	0.109798198
3	knn_withmeans	0.841993637	0.082859955	0.150872647
4	knn_withmeans	0.841993637	0.10445499	0.185853588
5	knn_withmeans	0.841993637	0.124500626	0.216925727
6	knn_withmeans	0.841993637	0.142217141	0.243333908
7	knn_withmeans	0.841993637	0.158381446	0.266612337
8	knn_withmeans	0.841993637	0.172790729	0.286738146
9	knn_withmeans	0.841993637	0.186156133	0.304901648
10	knn_withmeans	0.841993637	0.198039917	0.320659558

1	knn_withzscore	0.838812301	0.031390448	0.060516227
2	knn_withzscore	0.838812301	0.058542445	0.1094464
3	knn_withzscore	0.838812301	0.082500829	0.150226254
4	knn_withzscore	0.838812301	0.103906064	0.184907154
5	knn_withzscore	0.838812301	0.123300661	0.214997855
6	knn_withzscore	0.838812301	0.140962616	0.241363958
7	knn_withzscore	0.838812301	0.157187463	0.26476066
8	knn_withzscore	0.838812301	0.171581381	0.284888109
9	knn_withzscore	0.838812301	0.184848113	0.302938101
10	knn_withzscore	0.838812301	0.196722012	0.318700871

1	knn_baseline	0.920466596	0.035532665	0.068423968
2	knn_baseline	0.920466596	0.066264244	0.123628492
3	knn_baseline	0.920466596	0.092088451	0.167426637
4	knn_baseline	0.920466596	0.113949378	0.202793844
5	knn_baseline	0.920466596	0.132426986	0.231542141
6	knn_baseline	0.920466596	0.148135388	0.255200119
7	knn_baseline	0.920466596	0.161645886	0.27499847
8	knn_baseline	0.920466596	0.173108905	0.291412828
9	knn_baseline	0.920466596	0.183144676	0.305503505
10	knn_baseline	0.920466596	0.191877453	0.317557838

1	SVD	0.845174973	0.032078857	0.061811635
2	SVD	0.845174973	0.059178517	0.11061206
3	SVD	0.845174973	0.082220658	0.149862346
4	SVD	0.845174973	0.101507907	0.181247479
5	SVD	0.845174973	0.117657113	0.206559065
6	SVD	0.845174973	0.131509994	0.227604518
7	SVD	0.845174973	0.14355849	0.245429223
8	SVD	0.845174973	0.153952098	0.260460284
9	SVD	0.845174973	0.163165484	0.273525439
10	SVD	0.845174973	0.171592168	0.285267688

1	SVDpp	0.830328738	0.030385677	0.058625952
2	SVDpp	0.830328738	0.056425262	0.105669704
3	SVDpp	0.830328738	0.078877385	0.144068893
4	SVDpp	0.830328738	0.09781882	0.175019105
5	SVDpp	0.830328738	0.114602011	0.201405962
6	SVDpp	0.830328738	0.129049195	0.223380697
7	SVDpp	0.830328738	0.141737722	0.242141682
8	SVDpp	0.830328738	0.15257355	0.257779852
9	SVDpp	0.830328738	0.162084304	0.271224279
10	SVDpp	0.830328738	0.170781283	0.283294751

1	NMF	0.87592789	0.03344002	0.064420673
2	NMF	0.87592789	0.06226824	0.116270972
3	NMF	0.87592789	0.086879025	0.158078966
4	NMF	0.87592789	0.107786222	0.191952025
5	NMF	0.87592789	0.125490539	0.219529938
6	NMF	0.87592789	0.141071965	0.243006659
7	NMF	0.87592789	0.154261959	0.262325148
8	NMF	0.87592789	0.165492323	0.278387801
9	NMF	0.87592789	0.175884919	0.292946625
10	NMF	0.87592789	0.184435588	0.304711127

1	SVD	0.874867444	0.033049519	0.063692936
2	SVD	0.874867444	0.060669132	0.113469532
3	SVD	0.874867444	0.083544649	0.152524147
4	SVD	0.874867444	0.102893707	0.184131583
5	SVD	0.874867444	0.119705392	0.210595637
6	SVD	0.874867444	0.134199562	0.23270373
7	SVD	0.874867444	0.146558865	0.251059872
8	SVD	0.874867444	0.15705159	0.26629865
9	SVD	0.874867444	0.166209891	0.279348359
10	SVD	0.874867444	0.174293584	0.290677557

1	SVDpp	0.870625663	0.031560944	0.060913713
2	SVDpp	0.870625663	0.058537445	0.109699151
3	SVDpp	0.870625663	0.082330431	0.150435022
4	SVDpp	0.870625663	0.103054673	0.184294659
5	SVDpp	0.870625663	0.120330535	0.211437906
6	SVDpp	0.870625663	0.135715213	0.234825296
7	SVDpp	0.870625663	0.148738443	0.25407115
8	SVDpp	0.870625663	0.160616094	0.271200216
9	SVDpp	0.870625663	0.170830155	0.285617717
10	SVDpp	0.870625663	0.180038352	0.298375137

1	NMF	0.871686108	0.032141544	0.061997079
2	NMF	0.871686108	0.059442974	0.111296309
3	NMF	0.871686108	0.08290898	0.151416254
4	NMF	0.871686108	0.103085986	0.184368475
5	NMF	0.871686108	0.120883202	0.212322115
6	NMF	0.871686108	0.13605778	0.235376624
7	NMF	0.871686108	0.149171333	0.254747771
8	NMF	0.871686108	0.160305755	0.270808918
9	NMF	0.871686108	0.169986117	0.284493593
10	NMF	0.871686108	0.178294992	0.29603822


### OBSERVACIONES

Lo óptimo sería alcanzar el máximo valor posible, 1, de precisión (1ª columna) y de recall (2ª columna). Sin embargo, por lo general si una aumenta la otra disminuye. Por lo tanto, se busca maximizar la tercera columna, operación que pondera ambas. 
A mayor k en las pruebas, se escogen más recomendados, lo que implica que la intersección entre recomendados y relevantes aumenta. Por lo tanto, el recall crece a medida que se aumenta la k.

1. KNN_basic y KNN_baseline son los que presentan una mayor precisión, lo que se refleja en un menor recall respecto a los otros dos.

2. A k´s menores, KNN_basic y KNN_baseline tienen un mayor "f1", puesto que la diferencia de recall´s con el resto es mínima. Sin embargo, según aumenta el k, las diferencias entre los recall´s también crecen, y pasan a ser los que peor "f1" tienen 

3. KNN_baseline es el que mayor "f1" tiene (ligeramente), en general. Por ello, se puede concluir que a pesar de ser bastantes parejos los 4, este es el mejor.

4. KNN_basic a k´s más elevados es el que ofrece un peor rendimiento.

5. Hemos probado otros 3 recomendadores basados en factorización de matrices. Con los parámetros por defecto se obtienen peores resultados. Sin embargo, modificando dichos valores es posible alcanzar mejores ponderaciones, sin sobrepasar a los 4 primeros.
